# An Approach to Image Enhancement
Adarsh Shah, MTech (A.I.)

[adarshshah@iisc.ac.in](adarshshah@iisc.ac.in)

Inspired from [LLNet](https://www.sciencedirect.com/science/article/pii/S003132031630125X?casa_token=iWWVnisqhCkAAAAA:j5fneY7SDdkzJ2okhWA2N807cXYg57fY4CIpCbAD2tESbw2Mwf9buVbuJwCphbaphLiTTig-ioU:) and VGG16

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from skimage import img_as_float
from skimage.io import imread_collection, imread, imsave
from skimage.transform import resize
from skimage.metrics import structural_similarity
from skimage.color import rgb2hsv, hsv2rgb

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Conv2DTranspose, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.activations import relu
from tensorflow.keras.metrics import mean_squared_error

from matplotlib import pyplot as plt

### 1.0 Data Preparation

In [3]:
data = imread_collection(['../input/imageenhancement/dped/dped/iphone/training_data/iphone/*'],conserve_memory=False)
xtrain = list()
for i,file in enumerate(data.files[:7000]):
  xtrain.append(rgb2hsv(img_as_float(data.load_func(file))))

In [4]:
data = imread_collection(['../input/imageenhancement/dped/dped/iphone/training_data/canon/*'],conserve_memory=False)
ytrain = list()
for i,file in enumerate(data.files[:7000]):
  ytrain.append(rgb2hsv(img_as_float(data.load_func(file))))

In [5]:
data = imread_collection(['../input/imageenhancement/dped/dped/iphone/test_data/patches/iphone/*'],conserve_memory=False)
xtest = list()
for i,file in enumerate(data.files[:1000]):
  xtest.append(rgb2hsv(img_as_float(data.load_func(file))))

In [6]:
data = imread_collection(['../input/imageenhancement/dped/dped/iphone/test_data/patches/canon/*'],conserve_memory=False)
ytest = list()
for i,file in enumerate(data.files[:1000]):
  ytest.append(rgb2hsv(img_as_float(data.load_func(file))))

In [7]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
xtest = np.array(xtest)
ytest = np.array(ytest)

In [8]:
Xt = xtrain[:,:,:,2]
Yt = ytrain[:,:,:,2]
Xtest = xtest[:,:,:,2]
Ytest = ytest[:,:,:,2]

In [9]:
Xt = np.reshape(Xt,(7000,100,100,1))
Yt = np.reshape(Yt,(7000,100,100,1))
Xtest = np.reshape(Xtest,(1000,100,100,1))
Ytest = np.reshape(Ytest,(1000,100,100,1))

### 2.0 Illumination Map Estimator

In [10]:
x = Input(shape=(100,100,1))

layer11 = Conv2D(filters=128,kernel_size=(7,7),strides=(1,1),padding='same',activation=relu)(x)
layer12 = Conv2D(filters=32,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer11)
layer13 = Conv2D(filters=16,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer12)

layer16 = Conv2DTranspose(filters=32,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer13)
layer17 = Conv2DTranspose(filters=128,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer16)
layer18 = Conv2DTranspose(filters=3,kernel_size=(7,7),strides=(1,1),padding='same',activation=relu)(layer17)

layer21 = Conv2D(filters=128,kernel_size=(7,7),strides=(1,1),padding='same',activation=relu)(layer18)
layer22 = Conv2D(filters=32,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer21)
layer23 = Conv2D(filters=16,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer22)

layer26 = Conv2DTranspose(filters=32,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer23)
layer27 = Conv2DTranspose(filters=128,kernel_size=(3,3),strides=(2,2),padding='same',activation=relu)(layer26)
layer28 = Conv2DTranspose(filters=1,kernel_size=(7,7),strides=(1,1),padding='same',activation=relu)(layer27)

y = Reshape((100,100,1))(layer28)

In [11]:
llnet = Model(x,y)
llnet.summary()

In [12]:
def myloss(im1,im2):
  return 1-tf.image.ssim(im1,im2,1)

llnet.compile(optimizer=Adam(),loss=mean_squared_error,metrics=[mean_squared_error])

In [16]:
llnet.fit(x=Xt,y=Yt,validation_data=(Xtest,Ytest),batch_size=32,epochs=15)

### 3.0 Evaluation

In [22]:
ypredict = llnet.predict(Ytest)

In [19]:
sum = 0
for a,b in zip(ypredict,Ytest):
    sum += structural_similarity(np.reshape(a,(100,100)),np.reshape(b,(100,100)))
sum /= len(Ytest)
print(sum)

In [28]:
i = np.random.randint(len(ytest))
plt.subplot(131)
plt.imshow(hsv2rgb(xtest[i]),'brg')
plt.subplot(132)
plt.imshow(hsv2rgb(ytest[i]),'brg')
plt.subplot(133)
img = xtest[i]
img[:,:,2] = np.reshape(ypredict[i],(100,100))
plt.imshow(hsv2rgb(img),'brg')
plt.show()
plt.clf()

### 4.0 Test on Real Images

In [29]:
data = imread_collection(['../input/imageenhancement/dped/dped/iphone/test_data/full_size_test_images/*'],conserve_memory=False)

In [30]:
I = np.random.randint(len(data.files))
image = imread(data.files[I])

imsave(fname=f'./original_{I}.jpg',arr=image)
image = rgb2hsv(img_as_float(image))

P,Q,_ = image.shape
P_n = P + 100 - P%100
Q_n = Q + 100 - Q%100

image = resize(image,(P_n,Q_n))

for i in range(P_n//100):
    for j in range(Q_n//100):
        illum = image[100*i:100*(i+1),100*j:100*(j+1),2]
        illum = np.reshape(illum,(1,100,100,1))
        illum = llnet.predict(illum)
        illum = np.reshape(illum,(100,100))
        image[100*i:100*(i+1),100*j:100*(j+1),2] = illum

image = resize(image,(P,Q))
imsave(fname=f'./output_{I}.jpg',arr=hsv2rgb(image))